# Ookla Broadband Performance Data

This notebook downloads and explores Ookla open data for fixed and mobile broadband performance.

**Important**: Run the cells in order, starting with the configuration cell below.

In [1]:
# Enable IPython autoreload for modules
%load_ext autoreload
%autoreload 2

import pandas as pd 
# Load configuration
data_dir = None # TO silence Pylance, data_dir is defined by the cofig.py file. 
%run ../../../config.py

from lib import download_ookla_files, get_ookla_data, list_ookla_objects, build_files_df

Project root: /Users/max/proj/WirelessIncome
Data directory: /Users/max/proj/WirelessIncome/data


In [2]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import os
from urllib.parse import urlparse

# Create S3 client with no signing (for public buckets)
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# List parquet URIs from Ookla bucket
parquet_files = list_ookla_objects(s3)

# Build dataset of file paths with metadata
files_df = build_files_df(parquet_files)

files_df.head(20)

,path,service_type,year,quarter
0,s3://ookla-open-data/parquet/performance/type=...,fixed,2019,1
1,s3://ookla-open-data/parquet/performance/type=...,fixed,2019,2
2,s3://ookla-open-data/parquet/performance/type=...,fixed,2019,3
3,s3://ookla-open-data/parquet/performance/type=...,fixed,2019,4
4,s3://ookla-open-data/parquet/performance/type=...,fixed,2020,1
5,s3://ookla-open-data/parquet/performance/type=...,fixed,2020,2
6,s3://ookla-open-data/parquet/performance/type=...,fixed,2020,3
7,s3://ookla-open-data/parquet/performance/type=...,fixed,2020,4
8,s3://ookla-open-data/parquet/performance/type=...,fixed,2021,1
9,s3://ookla-open-data/parquet/performance/type=...,fixed,2021,2


In [3]:
files_df.iloc[0]


path            s3://ookla-open-data/parquet/performance/type=...
service_type                                                fixed
year                                                         2019
quarter                                                         1
Name: 0, dtype: object

In [4]:
files_df.year.value_counts()

year
2019    8
2020    8
2021    8
2022    8
2023    8
2024    8
2025    6
Name: count, dtype: int64

In [5]:
TARGET_YEAR = 2022
TARGET_QUARTERS = [1, 2, 3, 4]
SERVICE_TYPES = ["mobile", "fixed"]

downloads = []
for service in SERVICE_TYPES:
    selection = download_ookla_files(
        files_df,
        years=TARGET_YEAR,
        quarters=TARGET_QUARTERS,
        service_type=service,
        data_dir=data_dir,
        s3=s3,
    )
    selection["ookla_service_type"] = service
    downloads.append(selection)

downloaded = pd.concat(downloads, ignore_index=True)
print(
    f"Downloaded metadata for {len(downloaded)} files | "
    f"year={TARGET_YEAR}, quarters={TARGET_QUARTERS}, services={SERVICE_TYPES}"
)
downloaded.head()

Found 4 mobile files for years=[2022], quarters=[1, 2, 3, 4]
2022-01-01_performance_mobile_tiles.parquet already exists, skipping
2022-04-01_performance_mobile_tiles.parquet already exists, skipping
2022-07-01_performance_mobile_tiles.parquet already exists, skipping
2022-10-01_performance_mobile_tiles.parquet already exists, skipping

Downloaded 4 files
Found 4 fixed files for years=[2022], quarters=[1, 2, 3, 4]
Saved to /Users/max/proj/WirelessIncome/data/2022-01-01_performance_fixed_tiles.parquet
Saved to /Users/max/proj/WirelessIncome/data/2022-04-01_performance_fixed_tiles.parquet
Saved to /Users/max/proj/WirelessIncome/data/2022-07-01_performance_fixed_tiles.parquet
Saved to /Users/max/proj/WirelessIncome/data/2022-10-01_performance_fixed_tiles.parquet

Downloaded 4 files
Downloaded metadata for 8 files | year=2022, quarters=[1, 2, 3, 4], services=['mobile', 'fixed']


,path,service_type,year,quarter,local_path,ookla_service_type
0,s3://ookla-open-data/parquet/performance/type=...,mobile,2022,1,/Users/max/proj/WirelessIncome/data/2022-01-01...,mobile
1,s3://ookla-open-data/parquet/performance/type=...,mobile,2022,2,/Users/max/proj/WirelessIncome/data/2022-04-01...,mobile
2,s3://ookla-open-data/parquet/performance/type=...,mobile,2022,3,/Users/max/proj/WirelessIncome/data/2022-07-01...,mobile
3,s3://ookla-open-data/parquet/performance/type=...,mobile,2022,4,/Users/max/proj/WirelessIncome/data/2022-10-01...,mobile
4,s3://ookla-open-data/parquet/performance/type=...,fixed,2022,1,/Users/max/proj/WirelessIncome/data/2022-01-01...,fixed


In [6]:
# Load multiple quarters and both service types at once (ensure downloads exist first)
df = get_ookla_data(
    files_df=files_df,
    year=[TARGET_YEAR],
    quarter=TARGET_QUARTERS,
    service_type=SERVICE_TYPES,
    data_dir=data_dir,
)

df.head()

Loading 2022-01-01_performance_fixed_tiles.parquet...
  Loaded 6780815 rows
Loading 2022-04-01_performance_fixed_tiles.parquet...
  Loaded 6598700 rows
Loading 2022-07-01_performance_fixed_tiles.parquet...
  Loaded 6602066 rows
Loading 2022-10-01_performance_fixed_tiles.parquet...
  Loaded 6497987 rows
Loading 2022-01-01_performance_mobile_tiles.parquet...
  Loaded 3820724 rows
Loading 2022-04-01_performance_mobile_tiles.parquet...
  Loaded 4027744 rows
Loading 2022-07-01_performance_mobile_tiles.parquet...
  Loaded 4046154 rows
Loading 2022-10-01_performance_mobile_tiles.parquet...
  Loaded 3838065 rows
Concatenated 8 files -> 42212255 total rows


,quadkey,tile,tile_x,tile_y,avg_d_kbps,avg_u_kbps,avg_lat_ms,avg_lat_down_ms,avg_lat_up_ms,tests,devices,ookla_year,ookla_quarter,ookla_service_type
0,0022332203013331,"POLYGON((-162.603149414062 66.8999063523575, -...",-162.6004,66.8988,50519,24352,18,NaN,NaN,18,5,2022,1,fixed
1,0022332203013333,"POLYGON((-162.603149414062 66.8977510769183, -...",-162.6004,66.8967,10057,3988,9,NaN,NaN,2,1,2022,1,fixed
2,0022332203031110,"POLYGON((-162.608642578125 66.8955956114071, -...",-162.6059,66.8945,900297,75822,7,NaN,NaN,2,1,2022,1,fixed
3,0022332203031111,"POLYGON((-162.603149414062 66.8955956114071, -...",-162.6004,66.8945,370955,62117,8,NaN,NaN,4,3,2022,1,fixed
4,0022332203031113,"POLYGON((-162.603149414062 66.8934399558101, -...",-162.6004,66.8924,323725,37943,23,NaN,NaN,3,2,2022,1,fixed


In [7]:
# Inspect which metadata combinations are present after concatenation
t = (
    df[["ookla_year", "ookla_quarter", "ookla_service_type"]]
    .drop_duplicates()
    .sort_values(["ookla_service_type", "ookla_year", "ookla_quarter"])
)
t

,ookla_year,ookla_quarter,ookla_service_type
0,2022,1,fixed
6780815,2022,2,fixed
13379515,2022,3,fixed
19981581,2022,4,fixed
26479568,2022,1,mobile
30300292,2022,2,mobile
34328036,2022,3,mobile
38374190,2022,4,mobile


In [8]:
import geopandas as gpd
from shapely import wkb, wkt
from shapely.geometry import Point


def _load_geometry(value):
    if hasattr(value, "geom_type"):
        return value
    if isinstance(value, bytes):
        return wkb.loads(value)
    if isinstance(value, str):
        try:
            return wkt.loads(value)
        except Exception:
            pass
    return None

if "geometry" in df.columns:
    geometry_series = df["geometry"].apply(_load_geometry)
    missing_geometry = geometry_series.isna().sum()
    if missing_geometry:
        # Fall back to tile coordinates where geometry parsing failed
        fallback_points = [Point(x, y) for x, y in zip(df["tile_x"], df["tile_y"])]
        geometry_series = geometry_series.fillna(pd.Series(fallback_points, index=df.index))
else:
    geometry_series = [Point(x, y) for x, y in zip(df["tile_x"], df["tile_y"])]

gdf = gpd.GeoDataFrame(df.copy(), geometry=geometry_series, crs="EPSG:4326")
print(f"Created GeoDataFrame with {len(gdf)} points spanning {SERVICE_TYPES}")
gdf.head()

Created GeoDataFrame with 42212255 points spanning ['mobile', 'fixed']


,quadkey,tile,tile_x,tile_y,avg_d_kbps,avg_u_kbps,avg_lat_ms,avg_lat_down_ms,avg_lat_up_ms,tests,devices,ookla_year,ookla_quarter,ookla_service_type,geometry
0,0022332203013331,"POLYGON((-162.603149414062 66.8999063523575, -...",-162.6004,66.8988,50519,24352,18,NaN,NaN,18,5,2022,1,fixed,POINT (-162.6004 66.8988)
1,0022332203013333,"POLYGON((-162.603149414062 66.8977510769183, -...",-162.6004,66.8967,10057,3988,9,NaN,NaN,2,1,2022,1,fixed,POINT (-162.6004 66.8967)
2,0022332203031110,"POLYGON((-162.608642578125 66.8955956114071, -...",-162.6059,66.8945,900297,75822,7,NaN,NaN,2,1,2022,1,fixed,POINT (-162.6059 66.8945)
3,0022332203031111,"POLYGON((-162.603149414062 66.8955956114071, -...",-162.6004,66.8945,370955,62117,8,NaN,NaN,4,3,2022,1,fixed,POINT (-162.6004 66.8945)
4,0022332203031113,"POLYGON((-162.603149414062 66.8934399558101, -...",-162.6004,66.8924,323725,37943,23,NaN,NaN,3,2,2022,1,fixed,POINT (-162.6004 66.8924)


In [9]:
# Filter to continental US quadkeys and drop unused tile columns
point_gdf = gdf[gdf["quadkey"].str.startswith("0")].copy()
point_gdf = point_gdf.drop(columns=["quadkey", "tile_x", "tile_y", "tile"], errors="ignore")

print(f"Filtered to {len(point_gdf):,} Ookla samples after quadkey filter")
point_gdf.head()

Filtered to 14,074,789 Ookla samples after quadkey filter


,avg_d_kbps,avg_u_kbps,avg_lat_ms,avg_lat_down_ms,avg_lat_up_ms,tests,devices,ookla_year,ookla_quarter,ookla_service_type,geometry
0,50519,24352,18,NaN,NaN,18,5,2022,1,fixed,POINT (-162.6004 66.8988)
1,10057,3988,9,NaN,NaN,2,1,2022,1,fixed,POINT (-162.6004 66.8967)
2,900297,75822,7,NaN,NaN,2,1,2022,1,fixed,POINT (-162.6059 66.8945)
3,370955,62117,8,NaN,NaN,4,3,2022,1,fixed,POINT (-162.6004 66.8945)
4,323725,37943,23,NaN,NaN,3,2,2022,1,fixed,POINT (-162.6004 66.8924)


In [10]:
point_gdf.shape

(14074789, 11)

In [11]:
tracts_path = data_dir / "tracts_with_income_2022.parquet"
if not tracts_path.exists():
    raise FileNotFoundError(f"Missing census tract parquet at {tracts_path}. Run 01_census_tracts.ipynb first.")

tracts_gdf = gpd.read_parquet(tracts_path)[["GEOID", "geometry"]]
print(f"Loaded {len(tracts_gdf):,} census tracts from {tracts_path}")
tracts_gdf.head()

Loaded 83,225 census tracts from /Users/max/proj/WirelessIncome/data/tracts_with_income_2022.parquet


,GEOID,geometry
0,06077005127,"POLYGON ((-121.28708 37.83381, -121.28499 37.8..."
1,06077003406,"POLYGON ((-121.309 38.02824, -121.30461 38.028..."
2,06077004402,"POLYGON ((-121.27338 38.10811, -121.27286 38.1..."
3,06077001700,"POLYGON ((-121.26642 37.9753, -121.26453 37.97..."
4,06077000401,"POLYGON ((-121.31334 37.96416, -121.30553 37.9..."


In [12]:
# Spatial join Ookla samples to tracts in projected CRS
point_3857 = point_gdf.to_crs(epsg=3857)
tracts_3857 = tracts_gdf.to_crs(epsg=3857)

joined = gpd.sjoin(
    point_3857,
    tracts_3857,
    how="inner",
    predicate="within",
).drop(columns=["index_right"])

cols_to_keep = [
    "GEOID",
    "ookla_year",
    "ookla_quarter",
    "ookla_service_type",
    "avg_d_kbps",
    "tests",
]
joined = joined[cols_to_keep]

print(f"Joined {len(joined):,} Ookla samples to census tracts")
joined.head()

Joined 8,838,007 Ookla samples to census tracts


,GEOID,ookla_year,ookla_quarter,ookla_service_type,avg_d_kbps,tests
28370,53073011000,2022,1,fixed,22670,4
28374,53073011000,2022,1,fixed,89187,10
28375,53073011000,2022,1,fixed,35621,1
28376,53073011000,2022,1,fixed,31951,2
28377,53073011000,2022,1,fixed,93651,8


In [13]:
group_cols = ["GEOID", "ookla_year", "ookla_quarter", "ookla_service_type"]
metric_map = {"avg_d_kbps": "mean", "tests": "sum"}

df = (
    joined.groupby(group_cols)
    .agg(metric_map)
    .reset_index()
)

print(f"Aggregated to {len(df):,} tract-year-quarter-service rows")
df.head()

Aggregated to 652,829 tract-year-quarter-service rows


,GEOID,ookla_year,ookla_quarter,ookla_service_type,avg_d_kbps,tests
0,01001020100,2022,1,fixed,173362.210526,119
1,01001020100,2022,1,mobile,104696.142857,11
2,01001020100,2022,2,fixed,195308.857143,202
3,01001020100,2022,2,mobile,46461.400000,6
4,01001020100,2022,3,fixed,224019.571429,89


In [14]:
t = df.drop(columns=["ookla_year", "ookla_quarter"]).groupby(
    ["GEOID", "ookla_service_type"]
).agg({"avg_d_kbps": "mean", "tests": "sum"}).unstack(-1)

t.columns = [f"{metric}_{service}" for metric, service in t.columns]
t = t.reset_index()

t.head()


t.to_parquet(data_dir / "uss_mobile_fixed_2022.parquet")